In [1]:
import pandas as pd

In [2]:
url_dict = {'fantasy': 'https://www.footballdb.com/fantasy-football/index.html?pos=QB%2CRB%2CWR%2CTE&yr=2018&wk=all&rules=1',
            'QB': 'https://www.footballdb.com/stats/stats.html?mode=P&lg=NFL',
            'RB': 'https://www.footballdb.com/stats/stats.html?lg=NFL&yr=2018&type=reg&mode=R&conf=&limit=all',
            'WR/TE': 'https://www.footballdb.com/stats/stats.html?lg=NFL&yr=2018&type=reg&mode=C&conf=&limit=all'}

In [3]:
def add_stats(main_df, position):
    """Adds more player stats to the main fantasy stats dataframe.
    
    Args:
    main_df -- (pd.DataFrame) the dataframe you want to add stats to
    position -- (str) one of ['QB', 'RB', 'WR/TE']
    """
    url = url_dict[position]
    df = pd.read_html(url, header=0)[0]
    df['position'] = position

    name_splits = df['Player'].str.split('([a-z])[A-Z]\.', expand=True)
    df['player'] = name_splits.iloc[:, 0] + name_splits.iloc[:, 1]
    del df['Player']

    main_df = pd.merge(main_df, df, on='player', how='outer')
    
    coalesce_cols = [i for i in main_df.columns if '_x' in i]
    for col in coalesce_cols:
        main_df[col[:-2]] = main_df[col].combine_first(main_df[col[:-2]+'_y'])
        del main_df[col]
        del main_df[col[:-2]+'_y']
        
    return(main_df)

In [4]:
## Read in fantasy data
url = url_dict['fantasy']
stats = pd.read_html(url, header=1)[0].iloc[:, :-1]

cols = ['player', 'bye_wk', 'fantasy_pts', 'pass_att', 'pass_cmp', 'pass_yds', 'pass_td', 'pass_int', 'pass_2pt', 
        'rush_att', 'rush_yds', 'rush_td', 'rush_2pt', 'rec_rcp', 'rec_yds', 'rec_td', 'rec_2pt', 'fumbles_lost']
stats.columns = cols
cols = ['player', 'bye_wk', 'fantasy_pts','pass_2pt', 'rush_2pt','rec_2pt', 'fumbles_lost']
stats = stats[cols]

name_splits = stats['player'].str.split('([a-z])[A-Z]\.', expand=True)
stats['player'] = name_splits.iloc[:, 0] + name_splits.iloc[:, 1]

In [5]:
## Add stats
stats = add_stats(stats, 'QB')
stats = add_stats(stats, 'RB')
stats = add_stats(stats, 'WR/TE')

In [6]:
stats.shape

(428, 29)

In [7]:
stats.columns

Index(['player', 'bye_wk', 'fantasy_pts', 'pass_2pt', 'rush_2pt', 'rec_2pt',
       'fumbles_lost', 'Cmp', 'Pct', 'YPA', 'TD%', 'Int', 'Int%', 'Sack',
       'Loss', 'Rate', 'Att', 'Rec', 'Tar', 'YAC', 'Gms', 'Avg', 'YPG', 'FD',
       'Team', 'Yds', 'TD', 'Lg', 'position'],
      dtype='object')

In [8]:
stats

,player,bye_wk,fantasy_pts,pass_2pt,rush_2pt,rec_2pt,fumbles_lost,Cmp,Pct,YPA,...,YAC,Gms,Avg,YPG,FD,Team,Yds,TD,Lg,position
0,Patrick Mahomes,12.0,140.0,0.0,0.0,0.0,0.0,90.0,65.2,8.7,...,NaN,4.0,2.94,13.2,5.0,KC,1200.0,14.0,58t,QB
1,Matt Ryan,8.0,124.0,2.0,0.0,0.0,0.0,99.0,68.3,9.1,...,NaN,4.0,3.58,10.8,5.0,ATL,1316.0,10.0,75t,QB
2,Jared Goff,12.0,120.0,1.0,0.0,0.0,0.0,97.0,72.4,10.5,...,NaN,4.0,2.75,5.5,2.0,LA,1406.0,11.0,70t,QB
3,Ryan Fitzpatrick,5.0,120.0,0.0,0.0,0.0,0.0,87.0,67.4,10.5,...,NaN,4.0,3.44,15.5,4.0,TB,1356.0,11.0,75t,QB
4,Kirk Cousins,10.0,114.0,2.0,0.0,0.0,3.0,131.0,69.3,7.3,...,NaN,4.0,5.08,15.2,4.0,MIN,1387.0,10.0,75t,QB
5,Philip Rivers,8.0,111.0,2.0,0.0,0.0,0.0,100.0,68.0,7.9,...,NaN,4.0,0.29,0.5,1.0,LAC,1156.0,11.0,42t,QB
6,Drew Brees,6.0,111.0,1.0,0.0,0.0,0.0,122.0,75.8,8.0,...,NaN,4.0,0.62,1.2,3.0,NO,1295.0,8.0,42,QB
7,Deshaun Watson,10.0,103.0,0.0,0.0,0.0,1.0,92.0,62.2,8.4,...,NaN,4.0,6.71,40.2,10.0,HOU,1246.0,7.0,39t,QB
8,Ben Roethlisberger,7.0,102.0,2.0,0.0,0.0,2.0,119.0,64.0,7.6,...,NaN,4.0,2.30,5.8,4.0,PIT,1414.0,8.0,75t,QB
9,Andy Dalton,9.0,101.0,0.0,0.0,0.0,0.0,103.0,65.6,7.6,...,NaN,4.0,5.25,5.2,2.0,CIN,1197.0,11.0,49,QB


***

In [9]:
salaries = pd.read_csv('DKSalaries.csv')
salaries.shape

(74, 9)

In [10]:
pd.merge(stats[['player', 'fantasy_pts']], salaries[['Name', 'Salary']], left_on='player', right_on='Name', how='inner').shape

(48, 4)

***